In [162]:
from itertools import pairwise
lmap = lambda f,i : list(map(f, i))

In [163]:
with open("test_input.txt", "r") as f:
    test_input = f.read().split("\n\n")

with open("input.txt", "r") as f:
    real_input = f.read().split("\n\n")

test_input

['seeds: 79 14 55 13',
 'seed-to-soil map:\n50 98 2\n52 50 48',
 'soil-to-fertilizer map:\n0 15 37\n37 52 2\n39 0 15',
 'fertilizer-to-water map:\n49 53 8\n0 11 42\n42 0 7\n57 7 4',
 'water-to-light map:\n88 18 7\n18 25 70',
 'light-to-temperature map:\n45 77 23\n81 45 19\n68 64 13',
 'temperature-to-humidity map:\n0 69 1\n1 0 69',
 'humidity-to-location map:\n60 56 37\n56 93 4']

In [164]:
def get_seeds(sections):
    return lmap(int, sections[0].split()[1:])

get_seeds(test_input)

[79, 14, 55, 13]

In [165]:
def split_section(section):
    range_lines = section.split("\n")[1:]
    return [lmap(int, line.split()) for line in range_lines]

split_section(test_input[2])

[[0, 15, 37], [37, 52, 2], [39, 0, 15]]

In [166]:
def convert_to_ranges(split_section):
    return sorted([((s, s+l-1), (d, d+l-1)) for d,s,l in split_section], key=lambda r: r[0])

convert_to_ranges(split_section(test_input[2]))

[((0, 14), (39, 53)), ((15, 51), (0, 36)), ((52, 53), (37, 38))]

In [167]:
def find_range(id, ranges):
    possible_ranges = [((a1,a2), (b1, b2)) for ((a1,a2),(b1,b2)) in ranges if a1<=id<=a2]
    return possible_ranges[0] if possible_ranges else ((id, id), (id, id))

def convert_value(id, ranges):
    (s1, _), (d1, d2) = find_range(id, ranges)
    return id - s1 + d1

convert_value(79, convert_to_ranges(split_section(test_input[1])))

81

In [168]:
def get_sequential_ranges(sections):
    return [convert_to_ranges(split_section(section)) for section in sections[1:]]

get_sequential_ranges(test_input)

[[((50, 97), (52, 99)), ((98, 99), (50, 51))],
 [((0, 14), (39, 53)), ((15, 51), (0, 36)), ((52, 53), (37, 38))],
 [((0, 6), (42, 48)),
  ((7, 10), (57, 60)),
  ((11, 52), (0, 41)),
  ((53, 60), (49, 56))],
 [((18, 24), (88, 94)), ((25, 94), (18, 87))],
 [((45, 63), (81, 99)), ((64, 76), (68, 80)), ((77, 99), (45, 67))],
 [((0, 68), (1, 69)), ((69, 69), (0, 0))],
 [((56, 92), (60, 96)), ((93, 96), (56, 59))]]

In [169]:
def follow_ranges_list(id, ranges_list):
    for ranges in ranges_list:
        id = convert_value(id, ranges)
    return id

follow_ranges_list(13, get_sequential_ranges(test_input))

35

In [170]:
def get_score_1(sections):
    seeds = get_seeds(sections)
    sequential_ranges = get_sequential_ranges(sections)
    locations = [follow_ranges_list(seed, sequential_ranges) for seed in seeds]
    return min(locations)

get_score_1(test_input)

35

In [171]:
get_score_1(real_input)

323142486

In [172]:
def get_seed_ranges(sections):
    part_1_seeds = get_seeds(sections)
    range_starts = part_1_seeds[0::2]
    range_lengths = part_1_seeds[1::2]
    return [((s, s+l-1), (s, s+l-1)) for s,l in zip(range_starts, range_lengths)]

get_seed_ranges(test_input)

[((79, 92), (79, 92)), ((55, 67), (55, 67))]

In [173]:
def overlaps(range1, range2):
    do_swap = range1[0] > range2[0]
    lower_range = range2 if do_swap else range1
    upper_range = range1 if do_swap else range2
    return lower_range[1] >= upper_range[0]

overlaps((4,7), (1,4))

True

In [186]:
def clip_range_map_input(range, range_map):
    rl, ru = range
    (s1, e1), (s2, e2) = range_map
    new_lower = max(rl, s1)
    new_upper = min(ru, e1)
    new_length = new_upper - new_lower
    new_map_lower = s2 + (new_lower - s1)
    
    result = ((new_lower, new_upper), (new_map_lower, new_map_lower + new_length))
    if (new_lower<rl): print(f"{new_lower} lower than range {rl}")
    return result

clip_range_map_input((1, 1), ((0,5), (5,10)))

((1, 1), (6, 6))

In [222]:

def get_overlapping_range_maps(range_map_in, range_maps_out):
    return [clip_range_map_input(range_map_in[1], rm) for rm in range_maps_out if overlaps(range_map_in[1], rm[0])]

get_overlapping_range_maps(((55, 67), (57, 69)), [((0, 6), (42, 48)), ((7, 10), (57, 60)), ((11, 52), (0, 41)), ((53, 60), (49, 56))])

[((57, 60), (53, 56))]

In [194]:
def compose_range_map(range_map1, range_map2):
    # precondition: (e,f) subset of (c,d)
    ((a,b), (c,d)) = range_map1
    ((e,f), (g,h)) = range_map2
    start_delta = e-c
    length = f-e
    start = a+start_delta
    return ((start, start+length), (g, h))

compose_range_map(((1, 3), (2, 4)), ((2,4), (8,10)))

((1, 3), (8, 10))

In [219]:
def fill_range_map_input_gaps(range, range_maps):
    l,u = range
    range_maps.sort(key=lambda rm: rm[0][0])
    lower_map_bound = range_maps[0][0][0]
    upper_map_bound = range_maps[-1][0][1]
    
    gaps = [((ub1+1, lb2-1),)*2 for (_, (_, ub1)),((lb2, _), _) in pairwise(range_maps) if lb2-ub1>1]

    if lower_map_bound > l:
        gaps.append( ((l, lower_map_bound-1),)*2 )
    if upper_map_bound < u:
        gaps.append( ((upper_map_bound+1, u),)*2 )
    
    return sorted(range_maps + gaps, key=lambda rm: rm[0][0])

fill_range_map_input_gaps((0, 10), [((3,5), (18, 20)), ((0,2), (30,32))])

[((0, 2), (30, 32)), ((3, 5), (18, 20)), ((6, 10), (6, 10))]

In [223]:
def compose_maps(range_map, range_maps):
    overlapping_maps = get_overlapping_range_maps(range_map, range_maps)
    
    if not overlapping_maps: 
        return [range_map]

    overlapping_maps = fill_range_map_input_gaps(range_map[1], overlapping_maps)
    composition = [compose_range_map(range_map, rm) for rm in overlapping_maps]
    return composition

compose_maps(get_seed_ranges(test_input)[0], get_sequential_ranges(test_input)[0])
 

[((79, 92), (81, 94))]

In [228]:
def follow_maps(input_maps, section_maps):
    return [rm for input_map in input_maps for rm in compose_maps(input_map, section_maps)]

In [230]:
def compose_sections(sections):
    range_maps = get_seed_ranges(sections)
    for i, section_maps in enumerate(get_sequential_ranges(sections)):
        range_maps = follow_maps(range_maps, section_maps)
        if any(v < 0 for rm in range_maps for r in rm for v in r):
            print(f"alert, {i}")
    return range_maps

print(f"range maps have consistent range: {all(lb1-ub1==lb2-ub2 for ((lb1,ub1),(lb2,ub2)) in compose_sections(test_input))}")
compose_sections(real_input)

range maps have consistent range: True
alert, 3
alert, 4
alert, 5
alert, 6


[((-1090823547, -737193987), (241579313, 595208873)),
 ((-1012595171, -599020033), (138173954, 551749092)),
 ((-737193986, -599020033), (0, 138173953)),
 ((-599020032, -285837807), (3893533521, 4206715746)),
 ((-285837806, -43842316), (1339339293, 1581334783)),
 ((-1076033739, -624971479), (1426555548, 1877617808)),
 ((-624971478, -474312910), (1877617809, 2028276377)),
 ((-474312909, -432780764), (2028276378, 2069808523)),
 ((-1012595171, -599020033), (138173954, 551749092)),
 ((-893725315, -737193987), (438677545, 595208873)),
 ((-737193986, -599020033), (0, 138173953)),
 ((-599020032, -285837807), (3893533521, 4206715746)),
 ((-285837806, -235682532), (1339339293, 1389494567)),
 ((-432780763, -235682532), (241579313, 438677544)),
 ((-273356937, 252440709), (1351820162, 1877617808)),
 ((252440710, 403099278), (1877617809, 2028276377)),
 ((403099279, 478134793), (2028276378, 2103311892)),
 ((-235682531, -103787894), (2069808524, 2201703161)),
 ((-103787893, -54155810), (1289707209, 13